In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import string, re


In [2]:
dataset_path = "C:\\Users\\Admin\\PycharmProjects\\Tensorflow_learning\\datasets\\NYT_Comments\\"

In [3]:
import os
all_headlines = []
for filename in os.listdir(dataset_path):
    if 'Articles' in filename:
        article_df = pd.read_csv(dataset_path + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

In [5]:
len(all_headlines)

886

In [6]:
all_headlines = [line for line in all_headlines if line!= "Unknown"]
print(all_headlines[:10])

['Finding an Expansive View  of a Forgotten People in Niger', 'And Now,  the Dreaded Trump Curse', 'Venezuela’s Descent Into Dictatorship', 'Stain Permeates Basketball Blue Blood', 'Taking Things for Granted', 'The Caged Beast Awakens', 'An Ever-Unfolding Story', 'O’Reilly Thrives as Settlements Add Up', 'Mouse Infestation', 'Divide in G.O.P. Now Threatens Trump Tax Plan']


In [7]:
def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_headlines]
print(corpus[:10])

['finding an expansive view  of a forgotten people in niger', 'and now  the dreaded trump curse', 'venezuelas descent into dictatorship', 'stain permeates basketball blue blood', 'taking things for granted', 'the caged beast awakens', 'an everunfolding story', 'oreilly thrives as settlements add up', 'mouse infestation', 'divide in gop now threatens trump tax plan']


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku

tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to a token sequence 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(inp_sequences[:10])

[[169, 17], [169, 17, 665], [169, 17, 665, 367], [169, 17, 665, 367, 4], [169, 17, 665, 367, 4, 2], [169, 17, 665, 367, 4, 2, 666], [169, 17, 665, 367, 4, 2, 666, 170], [169, 17, 665, 367, 4, 2, 666, 170, 5], [169, 17, 665, 367, 4, 2, 666, 170, 5, 667], [6, 80]]


In [10]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [11]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # ----------Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    # ----------Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    # ----------Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [12]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 10)            24220     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2422)              244622    
                                                                 
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(predictors, label, epochs=300, verbose=1)

Epoch 1/300
151/151 [==============================] - 2s 14ms/step - loss: 2.5994
Epoch 2/300
151/151 [==============================] - 2s 13ms/step - loss: 2.5549
Epoch 3/300
151/151 [==============================] - 2s 14ms/step - loss: 2.5196
Epoch 4/300
151/151 [==============================] - 2s 13ms/step - loss: 2.4909
Epoch 5/300
151/151 [==============================] - 2s 14ms/step - loss: 2.4518
Epoch 6/300
151/151 [==============================] - 2s 15ms/step - loss: 2.4306
Epoch 7/300
151/151 [==============================] - 2s 14ms/step - loss: 2.4005
Epoch 8/300
151/151 [==============================] - 2s 15ms/step - loss: 2.3728
Epoch 9/300
151/151 [==============================] - 1s 10ms/step - loss: 2.3319
Epoch 10/300
151/151 [==============================] - 2s 10ms/step - loss: 2.3056
Epoch 11/300
151/151 [==============================] - 1s 8ms/step - loss: 2.2840
Epoch 12/300
151/151 [==============================] - 2s 12ms/step - loss: 2.2590
Ep

151/151 [==============================] - 2s 13ms/step - loss: 0.8248
Epoch 99/300
151/151 [==============================] - 2s 15ms/step - loss: 0.8158
Epoch 100/300
151/151 [==============================] - 2s 13ms/step - loss: 0.8020
Epoch 101/300
151/151 [==============================] - 2s 15ms/step - loss: 0.7901
Epoch 102/300
151/151 [==============================] - 3s 21ms/step - loss: 0.7887
Epoch 103/300
151/151 [==============================] - 2s 15ms/step - loss: 0.7704
Epoch 104/300
151/151 [==============================] - 2s 13ms/step - loss: 0.7675
Epoch 105/300
151/151 [==============================] - 2s 14ms/step - loss: 0.7537
Epoch 106/300
151/151 [==============================] - 2s 13ms/step - loss: 0.7599
Epoch 107/300
151/151 [==============================] - 2s 13ms/step - loss: 0.7588
Epoch 108/300
151/151 [==============================] - 2s 13ms/step - loss: 0.7469
Epoch 109/300
151/151 [==============================] - 2s 14ms/step - loss: 0.

151/151 [==============================] - 2s 13ms/step - loss: 0.3972
Epoch 195/300
151/151 [==============================] - 2s 12ms/step - loss: 0.3986
Epoch 196/300
151/151 [==============================] - 2s 12ms/step - loss: 0.4047
Epoch 197/300
151/151 [==============================] - 2s 13ms/step - loss: 0.3980
Epoch 198/300
151/151 [==============================] - 2s 11ms/step - loss: 0.4033
Epoch 199/300
151/151 [==============================] - 3s 19ms/step - loss: 0.3967
Epoch 200/300
151/151 [==============================] - 2s 13ms/step - loss: 0.3944
Epoch 201/300
151/151 [==============================] - 2s 15ms/step - loss: 0.4062
Epoch 202/300
151/151 [==============================] - 2s 16ms/step - loss: 0.3923
Epoch 203/300
151/151 [==============================] - 2s 13ms/step - loss: 0.3873
Epoch 204/300
151/151 [==============================] - 2s 13ms/step - loss: 0.3888
Epoch 205/300
151/151 [==============================] - 2s 12ms/step - loss: 0

151/151 [==============================] - 2s 13ms/step - loss: 0.3171
Epoch 291/300
151/151 [==============================] - 2s 16ms/step - loss: 0.3268
Epoch 292/300
151/151 [==============================] - 1s 8ms/step - loss: 0.3231
Epoch 293/300
151/151 [==============================] - 1s 8ms/step - loss: 0.3221
Epoch 294/300
151/151 [==============================] - 1s 10ms/step - loss: 0.3238
Epoch 295/300
151/151 [==============================] - 2s 14ms/step - loss: 0.3210
Epoch 296/300
151/151 [==============================] - 2s 13ms/step - loss: 0.3241
Epoch 297/300
151/151 [==============================] - 1s 8ms/step - loss: 0.3202
Epoch 298/300
151/151 [==============================] - 2s 12ms/step - loss: 0.3232
Epoch 299/300
151/151 [==============================] - 2s 11ms/step - loss: 0.3227
Epoch 300/300
151/151 [==============================] - 2s 11ms/step - loss: 0.3134


In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],              maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list),axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print (generate_text("india and pakistan", 3, model, max_sequence_len))
print (generate_text("president trump", 3, model, max_sequence_len))
print (generate_text("united states", 4, model, max_sequence_len))
print (generate_text("donald trump", 2, model, max_sequence_len))
print (generate_text("new york", 3, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))